# Web Scraping

## Introdução
Neste notebook, vamos fazer o webscrapping das imagens dos personagens de anime que serão usadas para treinar e testar a IA reconhecedora, gerando assim nosso dataset.

## Sumário
1. [Instalação de Bibliotecas](#Instalação-de-Bibliotecas)
2. [Inicialização](#Inicialização)
3. [Obtenção dos Personagens](#Obtenção-dos-Personagens)
4. [Obtenção das Imagens](#Obtenção-das-Imagens)

## Instalação de Bibliotecas

Primeiramente, se ainda não foi feito, instalaremos todas as bibliotecas necessárias para esse projeto:


In [ ]:
%pip install -r requirements.txt

## Inicialização

Agora importaremos as bibliotecas e também inicializaremos as constantes globais que serão usadas a seguir:
Será exigido a chave de API do Google Cloud e também o Search Engine ID do motor de busca customizado, eles serão necessários posteriormente. Caso não tenha não insira nada e pode prosseguir até onde for possível.

In [51]:
import json
import os
from io import BytesIO
from pathlib import Path

import requests
from bs4 import BeautifulSoup, ResultSet, Tag
from PIL import Image

CHAR_FOLDER = Path("Characters")
TEST_FOLDER = CHAR_FOLDER / Path("Test")
TRAIN_FOLDER = CHAR_FOLDER / Path("Train")
CHAR_LIST = Path("characters.txt")
LINKS_JSON = "links.json"
API_KEY = input("Chave API Google Cloud:")
SEARCH_ENGINE_ID = input("Custom Search Engine ID:")

## Obtenção dos Personagens

Aqui iremos obter a lista de todos os personagens principais que iremos treinar nossa IA, no caso iremos escolher os personagens do arco do Stardust Crusaders do anime JoJo's Bizarre Adventure, além disso iremos estruturar os diretórios para o salvamento dos arquivos.

### Obtenção da Lista de Nomes

Aqui iremos criar um arquivo `characters.txt` com a lista dos nomes dos personagens principais, utilizando o site do MyAnimesList como referência:

In [25]:
def get_characters(url):
    response = requests.get(url)
    parsed_html = BeautifulSoup(response.content, "html.parser")
    characters: ResultSet[Tag] = parsed_html.find_all(
        "h3", class_="h3_character_name"
    )
    with open(CHAR_LIST, "w", encoding="utf-8") as character_file:
        for character in characters:
            div_parent: Tag = character.parent.parent.parent
            character_role: Tag = div_parent.findChildren("div")[3]
            
            # Verificamos se o personagem é um personagem principal
            if character_role.text.strip() == "Main":
                
                # Como o nome vem no padrão oriental iremos redefinir para o padrão ocidental, caso necessário
                character = character.text
                if ", " in character:
                    name_split = character.split(", ")
                    character = name_split[1] + " " + name_split[0]
                print(character)
                print(character, file=character_file)

url = "https://myanimelist.net/anime/26055/JoJo_no_Kimyou_na_Bouken_Part_3__Stardust_Crusaders_2nd_Season/characters"
get_characters(url)

Muhammad Avdol
Dio Brando
Iggy
Joseph Joestar
Noriaki Kakyouin
Joutarou Kuujou
Jean-Pierre Polnareff


### Criação da Estrutura das Pastas

Agora iremos criar e estruturar as pastas, no qual iremos guardar e separar as imagens, criando uma pasta para cada personagem.

In [29]:
def create_directories():
    CHAR_FOLDER.mkdir(exist_ok=True)
    TEST_FOLDER.mkdir(exist_ok=True)
    TRAIN_FOLDER.mkdir(exist_ok=True)
    with open(CHAR_LIST, "r", encoding="utf-8") as characters:
        for character in characters:
            character_name = character.strip().replace(" ","-")
            pasta = TEST_FOLDER / character_name
            pasta.mkdir(exist_ok=True)
            pasta = TRAIN_FOLDER / character_name
            pasta.mkdir(exist_ok=True)

create_directories()

## Obtenção das Imagens

Neste trecho iremos obter as imagens de fato, e no caso iremos utilizar API do Google para a obtenção das imagens.

**Nota:** Da forma que a obtenção das imagens será apresentado à seguir, não é garantido que as imagens serão boas, portanto faz-se necessário uma revisão manual das imagens e/ou uma re-execução do código com *offset*.

### Criação da Lista de Links

Agora iremos criar um arquivo `links.json` que irá conter a lista com os links das imagens de todos os personagens separando por personagem.
No código a seguir é necessário a chave de API e o ID da Search Engine da *Custom Search API* do Google a ser usada para pesquisar.

In [ ]:
def save_links(links: dict):
    with open(LINKS_JSON, 'w') as json_file:
        json.dump(links, json_file, indent=4)

def load_links() -> dict:
    if os.path.exists(LINKS_JSON):
        with open(LINKS_JSON, 'r') as json_file:
            return json.load(json_file)
    else:
        return {}

def fetch_image_links(links: dict[str,list], query: str, api_key: str, cse_id: str, limit=10, offset=0):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        'q': query,
        'cx': cse_id,
        'key': api_key,
        'searchType': 'image',
        'start': offset + 1,
        'num': limit        # Máximo é 10
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        search_results = response.json()
        image_links = [item['link'] for item in search_results.get('items', [])]
        
        # Salva os links das imagens no dict
        if len(links[query]) > 0:
            links[query] = links[query] + image_links
        else:
            links[query] = image_links
        print(f"Links das primeiras {limit} imagens de {query} (com offset de {offset}) salvos com sucesso.")
    else:
        print(f"Falha ao acessar a API do Google Custom Search. Status Code: {response.status_code}")

links = load_links()
with open(CHAR_LIST,"r") as character_list:
    for line in character_list:
        query = line.strip()
        fetch_image_links(links, query,api_key=API_KEY,cse_id=SEARCH_ENGINE_ID) 
save_links(links)

### Download das Imagens

Agora iremos ler o arquivo `links.json` criado anteriomente e usar a biblioteca Pillow para baixar as imagens, separando 80% das imagens para treino e 20% para teste.

In [49]:
links = load_links()
def download_images(links: dict[str, list]):
    for character_name, links_list in links.items():
        pasta_treino = TRAIN_FOLDER / character_name.strip().replace(" ", "-")
        pasta_teste = TEST_FOLDER / character_name.strip().replace(" ", "-")

        # Separa as imagens na proporção de 80% para treino e 20% para teste
        split_index = int(len(links_list) * 0.8)
        treino_links = links_list[:split_index]
        teste_links = links_list[split_index:]

        # Salva as imagens de treino
        for i, link in enumerate(treino_links):
            response = requests.get(link)
            image = Image.open(BytesIO(response.content))
            image.save(pasta_treino / f"{character_name}_{i}.jpg")

        # Salva as imagens de teste
        for i, link in enumerate(teste_links):
            response = requests.get(link)
            image = Image.open(BytesIO(response.content))
            image.save(pasta_teste / f"{character_name}_{i}.jpg")

download_images(links)